In [3]:
import talib
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
from time import sleep
import MetaTrader5 as mt5
import pandas_ta as ta
from scipy.signal import savgol_filter, find_peaks

In [27]:
if not mt5.initialize(
    login=114999529, server="Exness-MT5Trial6", password="Mypassword$1234"
):
    print("initialize() failed, error code =", mt5.last_error())
    quit()

login = 114999529
password = "Mypassword$1234"
server = "Exness-MT5Trial6"
print(mt5.login(login, password, server))

# Global Order Var 
order = None
magic = None

    
def place_trade(pair ,trade, quantity, sl = 0, trg = 0):

# Place trade in MT5

    global order, magic

    try:

        magic = random.randint(10000, 999999)

        if trade == 'B':

            # Buy
            request = {
                "action": mt5.TRADE_ACTION_DEAL,
                "symbol": pair,
                "volume": float(quantity), 
                "type": mt5.ORDER_TYPE_BUY,
                "price": float(mt5.symbol_info_tick(pair).ask),
                "sl": float(sl), 
                "tp": float(trg), 
                "deviation": 20, 
                "magic": int(magic), 
                "comment": "python script buy",
                "type_time": mt5.ORDER_TIME_GTC,
                "type_filling": mt5.ORDER_FILLING_FOK,  # Change the filling mode here
            }

        elif trade == 'S':
            
            # Sell
            request = {
                "action": mt5.TRADE_ACTION_DEAL,
                "symbol": pair,
                "volume": float(quantity), 
                "type": mt5.ORDER_TYPE_SELL,
                "price": float(mt5.symbol_info_tick(pair).ask),
                "sl": float(sl), 
                "tp": float(trg), 
                "deviation": 20, 
                "magic": int(magic), 
                "comment": "python script sell",
                "type_time": mt5.ORDER_TIME_GTC,
                "type_filling": mt5.ORDER_FILLING_FOK,  # Change the filling mode here
            }

        order = mt5.order_send(request)
        print('Basic: ',order)
        print('Order: ', order.order)

    except Exception as e:
        print(f"Execute New Order, error in sending order: {e}")
        
def update_order(pair, sl = 0 ,trg = 0):

    # Update the sl or TRG in order

    positions = mt5.positions_get()

    for i in positions:

        if i[-3] == pair:
            # Magic
            magic = positions[0][6]
            # ticket
            order = positions[0][0]

            break

    request = {
    'action' : mt5.TRADE_ACTION_SLTP,  # Type of trade operation
    'position' : order, # Ticket of the position
    'symbol' : pair,  # Symbol
    'sl' : sl,  # Stop Loss of the position
    'tp' : trg,  # Take Profit of the position
    'magic' : magic  # MagicNumber of the position
    }

    # Send the request
    try:
        result = mt5.order_send(request)
        if result.retcode != mt5.TRADE_RETCODE_DONE:
            print("OrderSend error %d" % result.retcode)
        else:
            print(result)


    except Exception as e:
        print(f"Update Order error: {e}")


def close_trade(pair):

    positions = mt5.positions_get()

    for i in positions:

        if i[-3] == pair:
            # Magic
            # magic = positions[0][6]
            # ticket
            order = positions[0][0]

            mt5.Close(pair, ticket=order)

            break


True


In [6]:
place_trade(pair = 'BTCUSD' ,trade = 'B', quantity =  0.01)

Basic:  OrderSendResult(retcode=10009, deal=280981306, order=499030663, volume=0.01, price=37089.73, bid=37080.53, ask=37089.73, comment='python script bu', request_id=2954580902, retcode_external=0, request=TradeRequest(action=1, magic=620292, order=0, symbol='BTCUSD', volume=0.01, price=37089.73, stoplimit=0.0, sl=0.0, tp=0.0, deviation=20, type=0, type_filling=0, type_time=0, expiration=0, comment='python script buy', position=0, position_by=0))
Order:  499030663


In [24]:
update_order(pair = 'BTCUSD', sl = 36567.5, trg = 38066.55 )

OrderSendResult(retcode=10009, deal=0, order=0, volume=0.01, price=37091.28, bid=37091.28, ask=37100.270000000004, comment='', request_id=2954580903, retcode_external=0, request=TradeRequest(action=6, magic=620292, order=0, symbol='BTCUSD', volume=0.0, price=0.0, stoplimit=0.0, sl=36567.5, tp=38066.55, deviation=0, type=0, type_filling=0, type_time=0, expiration=0, comment='', position=499030663, position_by=0))


In [28]:
close_trade(pair = 'BTCUSD')

In [23]:
import MetaTrader5 as mt5
import random
import pandas as pd
from datetime import datetime

class Trade:

    def __init__(self):
        if not mt5.initialize(login=114999529, server="Exness-MT5Trial6", password="Mypassword$1234"):
            print("initialize() failed, error code =", mt5.last_error())
            quit()

        self.login = 114999529
        self.password = "Mypassword$1234"
        self.server = "Exness-MT5Trial6"
        print(mt5.login(self.login, self.password, self.server))

    def place_trade(self, pair, trade, quantity, sl, trg):
        # Ensure you are logged in before placing a trade
        if not mt5.login(self.login, self.password, self.server):
            print("Login failed")
            return

        self.magic = random.randint(10000, 999999)

        if trade == 'B':
            request = {
                "action": mt5.TRADE_ACTION_DEAL,
                "symbol": pair,
                "volume": quantity,
                "type": mt5.ORDER_TYPE_BUY,
                "price": mt5.symbol_info_tick(pair).ask,
                "sl": sl,
                "tp": trg,
                "deviation": 20,
                "magic": self.magic,
                "comment": "python script buy",
                "type_time": mt5.ORDER_TIME_GTC,
                "type_filling": mt5.ORDER_FILLING_IOC,
            }
        elif trade == 'S':
            request = {
                "action": mt5.TRADE_ACTION_DEAL,
                "symbol": pair,
                "volume": quantity,
                "type": mt5.ORDER_TYPE_SELL,
                "price": mt5.symbol_info_tick(pair).ask,
                "sl": sl,
                "tp": trg,
                "deviation": 20,
                "magic": self.magic,
                "comment": "python script sell",
                "type_time": mt5.ORDER_TIME_GTC,
                "type_filling": mt5.ORDER_FILLING_FOK,
            }

        try:
            self.order = mt5.order_send(request)
            if self.order.retcode != mt5.TRADE_RETCODE_DONE:
                print("OrderSend error %d" % self.order.retcode)
            else:
                print(self.order)

        except Exception as e:
            print(f"Execute New Order, error in sending order: {e}")

    def update_order(self, pair, sl, trg):
        request = {
            'action': mt5.TRADE_ACTION_SLTP,
            'position': self.order.order,
            'symbol': pair,
            'sl': sl,
            'tp': trg,
            'magic': self.magic
        }

        try:
            result = mt5.order_send(request)
            if result.retcode != mt5.TRADE_RETCODE_DONE:
                print("OrderSend error %d" % result.retcode)
            else:
                print(result)

        except Exception as e:
            print(f"Update Order error: {e}")
            
    def trader(self):
        
        self.df = pd.DataFrame(mt5.copy_rates_from(
                'BTCUSD', mt5.TIMEFRAME_M15, datetime.now(), 15
            ))
        
        print(self.df.head())

ob = Trade()
ob.place_trade('BTCUSD', 'B', 0.1, 0, 0)
# ob.trader()


True
Execute New Order, error in sending order: 'NoneType' object has no attribute 'retcode'


In [28]:
request = {
                "action": mt5.TRADE_ACTION_DEAL,
                "symbol": 'EURUSD',
                "volume": 0.1,
                "type": mt5.ORDER_TYPE_SELL,
                "price": mt5.symbol_info_tick('BTCUSD').ask,
                "sl": 0,
                "tp": 0,
                "deviation": 20,
                "magic": 234000,
                "comment": "python script sell",
                "type_time": mt5.ORDER_TIME_GTC,
                "type_filling": mt5.ORDER_FILLING_FOK,
            }


if not mt5.initialize(
            login=114999529, server="Exness-MT5Trial6", password="Mypassword$1234"
        ):
            print("initialize() failed, error code =", mt5.last_error())
            quit()

login = 114999529
password = "Mypassword$1234"
server = "Exness-MT5Trial6"
print(mt5.login(login, password, server))
        

order = mt5.order_send(request)
print(order)

True
None
